In [1]:
# Import required libraries
import requests  # For sending HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML documents
import random  # For random sampling
import pandas as pd  # For handling dataframes

In [2]:
# 1. Get all paper links for a specific year
def get_all_paper_links(year): 
    base_url = "https://icml.cc"  # Base URL for the ICML website
    papers_url = f"https://icml.cc/virtual/{year}/papers.html?filter=titles"  # URL for the paper list page of a specific year
    try:
        # Send HTTP GET request to fetch the paper list page content
        response = requests.get(papers_url)
        response.raise_for_status()  # Check if the request was successful (status code 200)
        
        # Use BeautifulSoup to parse the HTML document
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all paper links
        paper_links = []  # List to store paper links
        for a_tag in soup.find_all('a', href=True):  # Loop through all <a> tags
            href = a_tag['href']  # Get the href attribute value (link)
            if f"/virtual/{year}/poster/" in href:  # Filter links to paper details pages
                paper_links.append(base_url + href)  # Add the full link to the list

        print(f"Found {len(paper_links)} papers for year {year}.")  # Print the number of papers for the year
        return paper_links  # Return the list of paper links
    except requests.RequestException as e:  # Catch request exceptions
        print(f"Error fetching paper links for year {year}: {e}")  # Print error message
        return []  # Return an empty list

In [3]:
# 2. Randomly sample a specified number of paper links
def get_random_papers(paper_links, num_papers):
    
    if num_papers > len(paper_links):  # If the requested number exceeds the available number
        print(f"Warning: Only {len(paper_links)} papers available, but {num_papers} requested.")
        return paper_links  # Return all available links
    return random.sample(paper_links, num_papers)  # Randomly sample the specified number of links

In [4]:
# 3. Scrape detailed information for a single paper
def get_paper_details(paper_url, year):
    
    try:
        # Send HTTP GET request to fetch the paper details page content
        response = requests.get(paper_url)
        response.raise_for_status()  # Check if the request was successful

        # Use BeautifulSoup to parse the HTML document
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract paper title
        title = soup.find('h2', class_='main-title').text.strip()  # Find <h2> tag and extract text

        # Extract authors
        authors = soup.find('h3', class_='card-subtitle').text.strip()  # Find <h3> tag and extract text

        # Extract abstract
        abstract_div = soup.find('div', id='abstractExample')  # Find the <div> tag containing the abstract
        abstract = abstract_div.text.strip() if abstract_div else "Abstract not available"  # Extract abstract text, or return default if not available

        # Extract keywords
        keywords = [a.text.strip() for a in soup.find_all('a', class_='text-secondary')]  # Find all keyword tags and extract text

        # Return paper information in a dictionary
        return {
            "url": paper_url,  # Paper link
            "title": title,  # Paper title
            "authors": authors,  # Authors
            "keywords": ", ".join(keywords),  # Join keywords into a string
            "abstract": abstract,  # Abstract
            "year": year  # Paper year
        }
    except requests.RequestException as e:  # Catch request exceptions
        print(f"Error fetching details for {paper_url}: {e}")  # Print error message
        return None  # Return None

In [5]:
# 4. Calculate the total number of papers for each year
def get_total_papers_by_year(years):

    total_papers = {}  # Dictionary to store total number of papers for each year
    for year in years:  # Loop through each year
        paper_links = get_all_paper_links(year)  # Get all paper links for the year
        total_papers[year] = len(paper_links)  # Count and store the number of papers
        print(f"Total papers for {year}: {len(paper_links)}")  # Print the number of papers for the year
    return total_papers  # Return the dictionary of total paper counts


In [6]:
# 5. Sample papers based on proportion
def fetch_papers_by_proportion(total_papers, total_samples=500):

    # Calculate the total number of papers
    total_articles = sum(total_papers.values())  # Sum the number of papers across all years
    print(f"Total articles across all years: {total_articles}")  # Print total number of articles

    # Calculate the sample count for each year based on proportion
    sample_counts = {}  # Dictionary to store the sample count for each year
    for year, count in total_papers.items():  # Loop through each year's paper count
        proportion = count / total_articles  # Calculate the proportion for the year
        sample_counts[year] = round(proportion * total_samples)  # Calculate the sample count based on the proportion and round it
        print(f"Proportion for {year}: {proportion:.2f}, Sample count: {sample_counts[year]}")  # Print proportion and sample count

    # Ensure the total sample count equals total_samples
    while sum(sample_counts.values()) != total_samples:  # If the total sample count does not match
        diff = total_samples - sum(sample_counts.values())  # Calculate the difference
        year = max(sample_counts, key=lambda k: sample_counts[k])  # Find the year with the most samples
        sample_counts[year] += diff  # Adjust the sample count for that year
        print(f"Adjusting sample count for {year} by {diff} to balance total.")  # Print the adjustment information

    print("Final sample counts by year:", sample_counts)  # Print the final sample counts
    return sample_counts  # Return the dictionary of sample counts

In [7]:
# 6. Main function
def main():

    # Define the range of years
    years = range(2020, 2025)  # Years from 2020 to 2024

    # Get the total number of papers for each year
    total_papers = get_total_papers_by_year(years)

    # Calculate the sample count for each year based on proportion
    sample_counts = fetch_papers_by_proportion(total_papers, total_samples=500)

    # Scrape all papers for the selected years
    all_papers_data = []  # List to store data for all papers
    for year, num_papers in sample_counts.items():  # Loop through each year's sample count
        paper_links = get_all_paper_links(year)  # Get all paper links for the year
        random_paper_links = get_random_papers(paper_links, num_papers)  # Randomly sample the specified number of paper links
        for paper_url in random_paper_links:  # Loop through each sampled paper link
            print(f"Fetching details for: {paper_url}")  # Print the current paper link being fetched
            paper_details = get_paper_details(paper_url, year)  # Scrape paper details
            if paper_details:  
                all_papers_data.append(paper_details)  # Add the paper details to the list

    # Save the data to an Excel file
    df = pd.DataFrame(all_papers_data, columns=["url", "title", "authors", "keywords", "abstract", "year"])
    df.to_excel("/Users/alan/Desktop/海外申请/Coding test/EPIC-CHINA/Data files_Junbo/Data/ICML1.xlsx", index=False)
    print("Data saved to ICML1.xlsx")# Print success message

# Call the main function
if __name__ == "__main__":
    main()


Found 1084 papers for year 2020.
Total papers for 2020: 1084
Found 1185 papers for year 2021.
Total papers for 2021: 1185
Found 1233 papers for year 2022.
Total papers for 2022: 1233
Found 1866 papers for year 2023.
Total papers for 2023: 1866
Found 2635 papers for year 2024.
Total papers for 2024: 2635
Total articles across all years: 8003
Proportion for 2020: 0.14, Sample count: 68
Proportion for 2021: 0.15, Sample count: 74
Proportion for 2022: 0.15, Sample count: 77
Proportion for 2023: 0.23, Sample count: 117
Proportion for 2024: 0.33, Sample count: 165
Adjusting sample count for 2024 by -1 to balance total.
Final sample counts by year: {2020: 68, 2021: 74, 2022: 77, 2023: 117, 2024: 164}
Found 1084 papers for year 2020.
Fetching details for: https://icml.cc/virtual/2020/poster/6207
Fetching details for: https://icml.cc/virtual/2020/poster/5799
Fetching details for: https://icml.cc/virtual/2020/poster/6143
Fetching details for: https://icml.cc/virtual/2020/poster/6346
Fetching det